In [1]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import argparse
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import mean_squared_error
from keras.models import Model, Sequential
from keras.layers import *
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
from sklearn.model_selection import train_test_split

In [3]:
model_input=[0,1,2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
model_output=[3]
train_data='seoul_housing_integrated.csv'

In [4]:
train_dataX = pd.read_csv(train_data, usecols = model_input)
train_dataY = pd.read_csv('temp.csv')

In [25]:
train_dataX.head(5)


,법정동코드,전용면적(㎡),계약년월,층,건축년도,랜드마크,영화관,쇼핑센터,대형마트,백화점,지하철,차선수,순환도로,한강,공원,산,학교,교육특구,대학병원
0,1117010100,83.04,201405,4,2012,2.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,1.0,3.0,0.0,0.0
1,1117010100,40.99,201411,1,1973,2.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,1.0,3.0,0.0,0.0
2,1117010100,68.61,201407,5,2006,2.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,1.0,3.0,0.0,0.0
3,1117010100,76.27,201501,5,2006,2.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,1.0,3.0,0.0,0.0
4,1117010100,76.27,201503,1,2006,2.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,1.0,3.0,0.0,0.0


In [6]:
code_list = train_dataX['법정동코드']
code_list = code_list.drop_duplicates()
code = {}
cnt = 0 
for co in code_list:
    code[co]=cnt
    cnt = cnt + 1
    
X = train_dataX

In [7]:
X['나이'] = X['계약년월']/100 - X['건축년도']
X['나이'] = X['나이'].astype(int)

In [8]:
X = X.drop(columns=['계약년월', '건축년도'], axis=1)

In [9]:
idx = 0
for c in X['법정동코드']:
    try :
        #print(c)
        X.at[idx, '법정동코드'] = code[c]
        #print(code[c])
        idx = idx + 1
    except :
        continue

In [85]:
X = train_dataX

,법정동코드,전용면적(㎡),계약년월,층,건축년도,랜드마크,영화관,쇼핑센터,대형마트,백화점,지하철,차선수,순환도로,한강,공원,산,학교,교육특구,대학병원
0,0,83.04,201405,4,2012,2.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,1.0,3.0,0.0,0.0
1,0,40.99,201411,1,1973,2.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,1.0,3.0,0.0,0.0
2,0,68.61,201407,5,2006,2.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,1.0,3.0,0.0,0.0
3,0,76.27,201501,5,2006,2.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,1.0,3.0,0.0,0.0
4,0,76.27,201503,1,2006,2.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,1.0,3.0,0.0,0.0
5,0,163.27,201410,3,2004,2.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,1.0,3.0,0.0,0.0
6,0,168.41,201412,2,2004,2.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,1.0,3.0,0.0,0.0
7,0,166.56,201503,2,2004,2.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,1.0,3.0,0.0,0.0
8,0,166.56,201503,3,2004,2.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,1.0,3.0,0.0,0.0
9,0,83.69,201408,4,2005,2.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,1.0,3.0,0.0,0.0


In [45]:
X.head(5)

,법정동코드,전용면적(㎡),층,랜드마크,영화관,쇼핑센터,대형마트,백화점,지하철,차선수,순환도로,한강,공원,산,학교,교육특구,대학병원,나이
0,0,83.04,4,2.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,1.0,3.0,0.0,0.0,2
1,0,40.99,1,2.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,1.0,3.0,0.0,0.0,41
2,0,68.61,5,2.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,1.0,3.0,0.0,0.0,8
3,0,76.27,5,2.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,1.0,3.0,0.0,0.0,9
4,0,76.27,1,2.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,1.0,3.0,0.0,0.0,9


In [31]:
train_dataY['price'].head(5)

0    45700
1    16000
2    42000
3    49800
4    50000
Name: price, dtype: int64

In [10]:
Y = train_dataY['price']
Y = Y/1000
Y = Y.astype(int)

In [11]:
XX = X.drop(columns=['법정동코드'])

In [12]:
X_train, X_test , y_train, y_test = train_test_split(XX, Y, test_size = 0.2, random_state=123456)

In [13]:
np.random.seed(123456)

model = Sequential()

model.add(Dense(40, input_dim = X_train.shape[1], activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(1, activation = 'linear'))

model.compile(loss='mean_absolute_error', optimizer='adam')

Instructions for updating:
Colocations handled automatically by placer.


In [14]:
model.fit(X_train, y_train, batch_size=2048, epochs=300, verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/300
219070/219070 [==============================] - 0s 2us/step - loss: 25.3295
Epoch 2/300
219070/219070 [==============================] - 0s 1us/step - loss: 21.4724
Epoch 3/300
219070/219070 [==============================] - 0s 1us/step - loss: 19.3013
Epoch 4/300
219070/219070 [==============================] - 0s 1us/step - loss: 17.5918
Epoch 5/300
219070/219070 [==============================] - 0s 1us/step - loss: 16.9351
Epoch 6/300
219070/219070 [==============================] - 0s 1us/step - loss: 16.6744
Epoch 7/300
219070/219070 [==============================] - 0s 1us/step - loss: 16.4855
Epoch 8/300
219070/219070 [==============================] - 0s 1us/step - loss: 16.3062
Epoch 9/300
219070/219070 [==============================] - 0s 1us/step - loss: 16.1384
Epoch 10/300
219070/219070 [==============================] - 0s 1us/step - loss: 15.9732
Epoch 11/300
219070/219070 [==============================] -

219070/219070 [==============================] - 0s 1us/step - loss: 12.5961
Epoch 90/300
219070/219070 [==============================] - 0s 1us/step - loss: 12.5692
Epoch 91/300
219070/219070 [==============================] - 0s 1us/step - loss: 12.5628
Epoch 92/300
219070/219070 [==============================] - 0s 1us/step - loss: 12.5412
Epoch 93/300
219070/219070 [==============================] - 0s 1us/step - loss: 12.5285
Epoch 94/300
219070/219070 [==============================] - 0s 1us/step - loss: 12.5316
Epoch 95/300
219070/219070 [==============================] - 0s 1us/step - loss: 12.5320
Epoch 96/300
219070/219070 [==============================] - 0s 1us/step - loss: 12.4939
Epoch 97/300
219070/219070 [==============================] - 0s 1us/step - loss: 12.4791
Epoch 98/300
219070/219070 [==============================] - 0s 1us/step - loss: 12.4976
Epoch 99/300
219070/219070 [==============================] - 0s 1us/step - loss: 12.4411
Epoch 100/300
219070/21

219070/219070 [==============================] - 0s 1us/step - loss: 11.3938
Epoch 270/300
219070/219070 [==============================] - 0s 1us/step - loss: 11.3976
Epoch 271/300
219070/219070 [==============================] - 0s 1us/step - loss: 11.4092
Epoch 272/300
219070/219070 [==============================] - 0s 1us/step - loss: 11.3908
Epoch 273/300
219070/219070 [==============================] - 0s 1us/step - loss: 11.3856
Epoch 274/300
219070/219070 [==============================] - 0s 1us/step - loss: 11.3991
Epoch 275/300
219070/219070 [==============================] - 0s 1us/step - loss: 11.3910
Epoch 276/300
219070/219070 [==============================] - 0s 1us/step - loss: 11.3758
Epoch 277/300
219070/219070 [==============================] - 0s 1us/step - loss: 11.3709
Epoch 278/300
219070/219070 [==============================] - 0s 1us/step - loss: 11.3763
Epoch 279/300
219070/219070 [==============================] - 0s 1us/step - loss: 11.3684
Epoch 280/300

In [15]:
dataY_predict = model.predict(X_test)

In [29]:
labels = dataY_predict.astype(int)

In [35]:
df = pd.DataFrame(dataY_predict.astype(int))
df = df.drop_duplicates()

In [41]:
df2 = y_test.drop_duplicates()
df.append(df2)
df2=df2.drop_duplicates()

In [43]:
labels = np.array(df2)

In [44]:
import sklearn.metrics as sm
print(sm.mean_absolute_error(dataY_predict,y_test))
print(sm.mean_squared_error(dataY_predict,y_test))
print(sm.confusion_matrix(y_test, dataY_predict.astype(int),labels=labels))

11.429209335337374
379.48138627846043
[[10  1  3 ...  0  0  0]
 [ 5 34  0 ...  0  0  0]
 [ 2  0 10 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]


In [47]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False, title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    # print(cm)
    # classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [24]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from sklearn.utils.multiclass import unique_labels

font_path = '/Library/Fonts/NanumBarunGothic.otf'
font_name=fm.FontProperties(fname=font_path, size=12).get_name()
plt.rc('font', family=font_name)



%matplotlib inline

In [ ]:
plot_confusion_matrix(y_test, dataY_predict.astype(int), classes = labels)

Confusion matrix, without normalization
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [19]:
X_test.drop_duplicates()

,법정동코드,전용면적(㎡),층,랜드마크,영화관,쇼핑센터,대형마트,백화점,지하철,차선수,순환도로,한강,공원,산,학교,교육특구,대학병원,나이
121377,97,119.4700,9,0.00,0.0,0.0,1.0,0.0,1.000000,10.000000,0.000000,0.000000,1.000000,0.000000,3.000000,0.0,0.00,14
89345,67,84.8700,11,0.00,1.0,3.0,2.0,0.0,2.600000,7.600000,0.400000,0.000000,0.800000,0.000000,2.600000,0.0,0.20,32
225145,144,84.7900,30,2.50,1.0,1.0,1.0,1.0,2.000000,10.000000,1.000000,2.000000,1.500000,0.000000,1.500000,0.0,0.00,8
28080,45,84.5450,3,0.00,0.0,0.0,0.0,0.0,4.000000,6.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.0,0.00,23
230205,146,84.8280,1,0.00,0.0,0.0,0.0,0.0,1.500000,8.000000,0.000000,0.500000,0.500000,0.000000,3.000000,0.0,0.00,33
186282,126,121.6600,2,0.75,2.0,0.0,0.0,0.0,2.500000,8.000000,0.750000,0.250000,1.250000,0.250000,3.250000,0.0,0.00,17
1765,13,66.9700,5,0.00,0.0,0.0,0.0,0.0,3.000000,6.000000,0.000000,0.000000,1.000000,0.000000,2.000000,0.0,0.00,43
129214,108,84.7500,8,0.00,0.0,0.0,1.0,1.0,1.750000,7.000000,0.000000,0.000000,1.000000,0.000000,2.500000,0.0,0.25,23
156370,118,61.7400,14,0.00,1.0,0.0,2.0,1.0,0.888889,8.888889,0.333333,0.000000,0.777778,0.333333,3.000000,0.0,0.00,29
262431,160,84.8600,13,0.00,0.0,0.0,0.0,0.0,1.000000,10.000000,0.000000,0.000000,0.500000,0.000000,4.500000,0.0,0.00,19


In [15]:
from sklearn.ensemble import RandomForestClassifier

In [16]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [18]:

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
seoul_rf = RandomForestClassifier(n_estimators=50, criterion='entropy', oob_score = True, random_state=123456)
seoul_rf.fit(X_train, y_train)

predicted = seoul_rf.predict(X_test)
accuracy = accuracy_score(y_test, predicted)

print(f'Out-of-bag score estimate: {seoul_rf.oob_score_:.3}')
print(f'Mean accuracy score: {accuracy:.3}')

print("특성:", X_train.columns)
print("특성 중요도:\n{}".format(seoul_rf.feature_importances_))


Out-of-bag score estimate: 0.134
Mean accuracy score: 0.135
특성: Index(['법정동코드', '전용면적(㎡)', '층', '랜드마크', '영화관', '쇼핑센터', '대형마트', '백화점', '지하철',
       '차선수', '순환도로', '한강', '공원', '산', '학교', '교육특구', '대학병원', '나이'],
      dtype='object')
특성 중요도:
[0.03655477 0.30154894 0.26295046 0.00586641 0.00516469 0.01017849
 0.0082677  0.00381581 0.01725359 0.01597922 0.00759492 0.01713376
 0.01262597 0.00595142 0.01345019 0.00812562 0.00439442 0.26314363]


In [19]:
import xgboost as xgb


In [21]:
seoul_xgb = xgb.XGBClassifier(max_depth=3,
                              learning_rate=0.02,
                              n_estimators=30,
                              gamma=0.1,
                              booster='gbtree')
seoul_xgb.fit(X_train, y_train)
#    y_pred = movie_xgb.predict(y_train)
print("훈련 세트 정확도: {:.3f}".format(seoul_xgb.score(X_train, y_train)))
# y_pred = movie_xgb.predict(y_test)
print("테스트 세트 정확도: {:.3f}".format(seoul_xgb.score(X_test, y_test)))

KeyboardInterrupt: 